In [ ]:
import cv2
import numpy as np
import csv
import pandas as pd
import matplotlib as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import chardet
from skimage.feature import local_binary_pattern
import copy
demoindex = 1

## read in images
* imgs: images
* cor: coordinate of the center of nerves
* contours: contours of nerves
* imgwithoutline: images outlined with nerve contours
* masks: ground truths
* pospatches: patches centered at nerve
* poswithoutline: centered nerve patches with nerve outline

For now, I'm not sure if we should use this centered nerve patch, possibly not.

In [ ]:
info = pd.read_csv("/kaggle/input/ultrasound-nerve-segmentation/train_masks.csv") 
image_path = "/kaggle/input/ultrasound-nerve-segmentation/train/{}_{}.tif"
mask_path = "/kaggle/input/ultrasound-nerve-segmentation/train/{}_{}_mask.tif"
rows = 420
cols = 580
window_height = 120
window_width = 140
step_size = [window_height, window_width]
label = []
features = []
nums = 120
li = []
i = 0
while i < nums:  #note the index of images that have nerve
    mask = cv2.imread(mask_path.format(info['subject'][i], info['img'][i]), 2)
    if (np.sum(mask) > 0):
        li.append(i)
    i = i + 1    
imgs = np.zeros((nums, rows, cols))
masks = np.zeros((nums, rows, cols))
contours = np.zeros((nums, rows, cols))
imgwithoutline = np.zeros((nums, rows, cols))
cor = np.zeros((nums, 2))
pospatch = np.zeros((nums, window_height, window_width))
poswithoutline = np.zeros((nums, window_height, window_width))


In [ ]:
for n in range(0,nums):
    img = cv2.imread(image_path.format(info['subject'][n], info['img'][n]), 2)
    mask = cv2.imread(mask_path.format(info['subject'][n], info['img'][n]), 2)
    imgm=cv2.medianBlur(img,5)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    equ = clahe.apply(imgm)
    imgs[n,:,:] = equ
    masks[n,:,:] = mask.astype(np.bool)
    imgwithoutline[n,:,:] = copy.deepcopy(equ)
    mask_outline = cv2.blur(mask, (3,3))
    mask_outline = mask_outline * ((mask_outline < 255) & (mask_outline > 0))
    contours[n,:,:] = mask_outline > 0
    temp = np.where(mask.astype(np.bool))
#     if temp[0].size != 0:         # center of the nerve
#         xs = temp[0]
#         ys = temp[1]
#         x = int((min(xs) + max(xs)) / 2)
#         y = int((min(ys) + max(ys)) / 2)
#         cor[n,0] = x
#         cor[n,1] = y
#         pospatch[n,:,:] = img[x - int(window_height / 2) : x + int(window_height / 2),y - int(window_width / 2):y + int(window_width / 2)]
#         temp = copy.deepcopy(pospatch[n,:,:])
#         contour = contours[n,x - int(window_height / 2) : x + int(window_height / 2),y - int(window_width / 2):y + int(window_width / 2)]
#         temp[contour>0] = 255
#         poswithoutline[n] = temp
#         imgwithoutline[n][contours[n] > 0]=255
imgwithmask = imgs * masks

In [ ]:
neg = []
allpatch = []
patcheswithoutline = []
labels = []
window_height = 70
window_width = 58
step_size = [window_height, window_width]
for n in range(0, nums):
    curneg = []
    curpatch = []
    curoutline = []
    img = imgs[n]
    mask = masks[n]
    withline = imgwithoutline[n]
    for i in range(0, rows - window_height + 1, step_size[0]):
        for j in range(0, cols - window_width + 1, step_size[1]):
            temp = img[i:i+window_height, j: j + window_width]
            m = mask[i:i+window_height, j:j+window_width]
            curpatch.append(temp)
            if (np.sum(m) == 0):    
                curneg.append(temp)
                labels.append(0)
            else:
                labels.append(1)
            temp = withline[i:i + window_height, j:j + window_width]
            curoutline.append(temp)
    curneg = np.stack(curneg)
    curpatch = np.stack(curpatch)
    curoutline = np.stack(curoutline)
    neg.append(curneg)
    allpatch.append(curpatch)
    patcheswithoutline.append(curoutline)
neg = np.array(neg)
allpatch = np.array(allpatch)
labels = np.array(labels, dtype = np.bool)
patcheswithoutline = np.array(patcheswithoutline)

In [ ]:
f, axarr = plt.subplots(6,10, figsize = (10 * 4, 6 * 4))
plt.subplots_adjust(wspace= 0.01, hspace=0.01)
patches = patcheswithoutline[2]
for i in range(0, 60):
    row = int(i / 10)
    col = int(i % 10)
    axarr[row][col].imshow(patches[i],cmap = 'gray')
    axarr[row][col].axis('off')
    

In [ ]:
f,axarr=plt.subplots(1,3,figsize = (18, 4))
axarr[0].imshow(imgs[li[demoindex]],cmap = 'gray')
axarr[0].set_title("image")
axarr[1].imshow(imgwithoutline[li[demoindex]],cmap = 'gray')
axarr[1].set_title("image withmask outline")
axarr[2].imshow(masks[li[demoindex]],cmap = 'gray')
axarr[2].set_title("mask")
plt.show()

# f,axarr=plt.subplots(1,3,figsize = (18, 4))
# axarr[0].imshow(pos[li[demoindex]],cmap = 'gray')
# axarr[0].set_title("centered")
# axarr[1].imshow(poswithoutline[li[demoindex]],cmap = 'gray')
# axarr[1].set_title("centered with contour")
# axarr[2].imshow(contours[demoindex,int(cor[li[demoindex],0] - window_height / 2) : int(cor[li[demoindex],0] + window_height / 2),int(cor[li[demoindex],1] - window_width / 2):int(cor[li[demoindex],1] + window_width / 2)],cmap = 'gray')
# axarr[2].set_title("contour")
# plt.show()

In [ ]:
l = 0
for i in range(0, nums):
    l = l + neg[i].shape[0]
negpatch = np.zeros((l, window_height, window_width))
cur = 0
for i in range(0, neg.shape[0]):
    curneg = neg[i]
    for j in range(0, curneg.shape[0]):
        negpatch[cur] = curneg[j,:,:]
        cur = cur + 1

## Form the training set
concatenate positive and negative together

In [ ]:
allpatch.shape
al = allpatch.reshape((60*nums,70,58))

In [ ]:
from scipy.stats import kurtosis,skew
features=[]
for i in range(0, al.shape[0]):
    vector = np.zeros(6)
    cur = al[i].flatten()
    m0 = np.mean(cur)
    m1 = np.mean(cur - m0)
    m2 = np.std(cur)
    m3 = np.sum(((cur-m0)**3)/(m2**3))/cur.size
    m4 = np.sum((cur-m0)**4/(m2**4))/cur.size
    m5 = m2 / m0
    vector[0] = m0
    vector[1] = m1
    vector[2] = m2
    vector[3] = m3
    vector[4] = m4
    vector[5] = m5
    features.append(vector)
features = np.stack(features)

In [ ]:
scores = np.zeros(features.shape[1])
for i in range(features.shape[1]):
    f = features[:, i]
    fa = f[labels == 1]
    fb = f[labels == 0]
    mi = np.mean(f)
    mia = np.mean(fa)
    mib = np.mean(fb)
    na = fa.shape[0]
    nb = fb.shape[0]
    va = np.var(fa)
    vb = np.var(fb)
    scores[i] = (na * np.square((mia - mi))+nb * np.square((mib-mi)))/(na*va+nb*vb)
indexes = np.argsort(scores) 

In [ ]:
scores

In [ ]:
from sklearn import svm
import joblib
filename = 'finalized_model.sav'
clf = svm.SVC(kernel="linear", gamma="scale", C=5.0)
clf.fit(features, labels)
joblib.dump(clf, filename)

In [ ]:
posf = features[labels == True]
negf = features[labels == False]
f, axarr = plt.subplots(6, 1, figsize = (4*2, 6*4))
axarr[0].hist(posf[:, 0], color = 'r', bins = 20, density = True)
axarr[0].hist(negf[:, 0], color = 'b', bins = 20, density = True,alpha = 0.5)
axarr[0].set_title('histogram of m0')

axarr[1].hist(posf[:, 1], color = 'r',bins = 20, density = True)
axarr[1].hist(negf[:, 1], color = 'b',bins = 20, density = True,alpha = 0.5)
axarr[1].set_title('histogram of m1')

axarr[2].hist(posf[:, 2], color = 'r', bins = 20, density = True)
axarr[2].hist(negf[:, 2], color = 'b', bins = 20, density = True,alpha = 0.5)
axarr[2].set_title('histogram of m2')

axarr[3].hist(posf[:, 3], color = 'r', bins = 20, density = True)
axarr[3].hist(negf[:, 3], color = 'b', bins = 20, density = True,alpha = 0.5)
axarr[3].set_title('histogram of m3')

axarr[4].hist(posf[:, 4], color = 'r', bins = 20, density = True)
axarr[4].hist(negf[:, 4], color = 'b', bins = 20, density = True,alpha = 0.5)
axarr[4].set_title('histogram of m4')

axarr[5].hist(posf[:, 5], color = 'r', bins = 20, density = True)
axarr[5].hist(negf[:, 5], color = 'b', bins = 20, density = True,alpha = 0.5)
axarr[5].set_title('histogram of m5')
plt.show()

## Form test set

In [ ]:
testnums = 30
testpatches = []
testlabels = []
for n in range(nums, nums + testnums):
    img = cv2.imread(image_path.format(info['subject'][n], info['img'][n]), 2)
    mask = cv2.imread(mask_path.format(info['subject'][n], info['img'][n]), 2)
    imgm=cv2.medianBlur(img,5)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    equ = clahe.apply(imgm)
    for i in range(0, rows - window_height + 1, step_size[0]):
        for j in range(0, cols - window_width + 1, step_size[1]):
            temp = imgm[i:i+window_height, j: j + window_width]
            m = mask[i:i+window_height, j:j+window_width]
            if (np.sum(m) > 0):
                testlabels.append(1)
            else:
                testlabels.append(0)
            testpatches.append(temp)
testpatches = np.array(testpatches)
testlabels = np.array(testlabels, dtype = np.bool)
testfeatures = []
for i in range(0, testpatches.shape[0]):
    vector = np.zeros(6)
    cur = al[i].flatten()
    m0 = np.mean(cur)
    m1 = np.mean(cur - m0)
    m2 = np.std(cur)
    m3 = np.sum(((cur-m0)**3)/(m2**3))/cur.size
    m4 = np.sum((cur-m0)**4/(m2**4))/cur.size
    m5 = m2 / m0
    vector[0] = m0
    vector[1] = m1
    vector[2] = m2
    vector[3] = m3
    vector[4] = m4
    vector[5] = m5
    testfeatures.append(vector)
testfeatures = np.stack(testfeatures)

In [ ]:
from sklearn.metrics import confusion_matrix
output = clf.predict(testfeatures)
confusion = confusion_matrix(testlabels, output)

In [ ]:
print(confusion)

## Feature Selection(I assume each sliding window's feature is given in a one-dimension vector)

## Throw the features and labels to svm, learn the possible location of nerve....

In [ ]:
scores = np.zeros(features.shape[1])
for i in range(features.shape[1]):
    f = features[:, i]
    fa = f[labels == 1]
    fb = f[labels == 0]
    mi = np.mean(f)
    mia = np.mean(fa)
    mib = np.mean(fb)
    na = fa.shape[0]
    nb = fb.shape[0]
    va = np.var(fa)
    vb = np.var(fb)
    scores[i] = (na * np.square((mia - mi))+nb * np.square((mib-mi)))/(na*va+nb*vb)
indexes = np.argsort(scores) 
the returned indexes are sorted by score in ascending order 
use this index to select features

In [ ]:
# L5 = np.array((1,4,6,4,1))
# L5 = L5.reshape(1,5)
# E5 = np.array((-1,-2,0,2,1))
# E5 = E5.reshape(1,5)
# S5 = np.array((-1,0,2,0,-1))
# S5 = S5.reshape(1,5)
# R5 = np.array((1,-4,6,-4,1))
# R5 = R5.reshape(1,5)
# kernel1 = np.kron(E5.T, L5)
# kernel2 = np.kron(R5.T, L5)
# kernel3 = np.kron(S5.T,E5)
# kernel4 = np.kron(S5.T,S5)
# kernel5 = np.kron(R5.T,R5)
# kernel6 = np.kron(L5.T,S5)
# kernel7 = np.kron(E5.T,E5)
# kernel8 = np.kron(E5.T,R5)
# kernel9 = np.kron(S5.T, R5)

In [ ]:
# n=0
# img = cv2.imread(image_path.format(info['subject'][n], info['img'][n]), cv2.CV_8U)
# mask = cv2.imread(mask_path.format(info['subject'][n], info['img'][n]), 2)
# from scipy import signal
# fig, axarr = plt.subplots(3, 3, figsize = (15,15))
# map1 = signal.convolve2d(img, kernel1, mode = 'same')
# axarr[0,0].imshow(map1,cmap = 'gray')
# map2 = signal.convolve2d(img, kernel2, mode = 'same')
# axarr[0,1].imshow(map2,cmap = 'gray')
# map3 = signal.convolve2d(img, kernel3, mode = 'same')
# axarr[0,2].imshow(map3,cmap = 'gray')
# map4 = signal.convolve2d(img, kernel4, mode = 'same')
# axarr[1,0].imshow(map4,cmap = 'gray')
# map5 = signal.convolve2d(img, kernel5, mode = 'same')
# axarr[1,1].imshow(map5,cmap = 'gray')
# map6 = signal.convolve2d(img, kernel6, mode = 'same')
# axarr[1,2].imshow(map6,cmap = 'gray')
# map7 = signal.convolve2d(img, kernel7, mode = 'same')
# axarr[2,0].imshow(map7,cmap = 'gray')
# map8 = signal.convolve2d(img, kernel8, mode = 'same')
# axarr[2,1].imshow(map8,cmap = 'gray')
# map9 = signal.convolve2d(img, kernel9, mode = 'same')
# axarr[2,2].imshow(map9,cmap = 'gray')

In [ ]:
# #test 
# #This model doesn't work. At least for now...
# k= 49
# testimg = cv2.imread(image_path.format(info['subject'][k], info['img'][k]), cv2.CV_8U)
# testmask = cv2.imread(mask_path.format(info['subject'][k], info['img'][k]), 2)
# testmask = testmask / 255
# testimgm=cv2.medianBlur(testimg,3)
# testequ = cv2.equalizeHist(testimgm)
# testlbp1=local_binary_pattern(testequ,8,1,'ror')
# testlbp2=local_binary_pattern(testequ,8,1,'uniform')
# testlabel = []
# testfeatures = []
# for i in range(0, rows - window_height + 1, step_size[0]):
#     for j in range(0, cols - window_width + 1, step_size[1]):
#         feature1 = testlbp1[i:i+window_height, j: j + window_width]
#         feature2 = testlbp2[i:i+window_height, j: j + window_width]
#         feature3 = testimg[i:i+window_height, j: j + window_width]
#         h1 = np.histogram(feature1,bins = range(0,257))
#         h2 = np.histogram(feature2,bins = range(0,257))
#         h3 = np.histogram(feature3,bins = range(0,257))
#         f = np.array((h1[0], h2[0],h3[0]))
#         maskpart = testmask[i:i+window_height, j: j + window_width]
#         testfeatures.append(f.flatten())
#         if (np.sum(maskpart) > 0):
#             testlabel.append(1)
#         else:
#             testlabel.append(0)
# testlabel = np.array(testlabel)
# testfeatures = np.array(testfeatures)
# testlab= clf.predict_proba(testfeatures)
# img = np.zeros((rows, cols),np.uint8)
# positive = testlab[:,0]
# for i in range(0, rows - window_height + 1, step_size[0]):
#     for j in range(0, cols - window_width + 1, step_size[1]):
#         a = int(i / step_size[0])
#         b = int(j / step_size[1])
#         start = (j,i)
#         end = (j+ window_width,i+window_height)
#         if (positive[a * int((cols - window_width + 1)/step_size[1]+1) + b] > 0.9):
#             img = cv2.rectangle(img,start,end,255,-1)
# plt.imshow(img, cmap = 'gray')
# img = img / 255


In [ ]:
# brg = cm.get_cmap('brg', 256)
# newcolors = brg(np.linspace(0, 1, 256))
# black = np.array([0, 0, 0, 1])
# green = np.array([0, 1, 0, 1])
# blue = np.array([0, 0, 1, 1])
# red = np.array([1, 0, 0, 1])
# newcolors[0:62, :] = black
# newcolors[62:126, :] = red
# newcolors[190:, :] = green
# newcolors[126:190,:] = blue
# newcmp = ListedColormap(newcolors)
# c=(np.array(img)*np.array(testmask))*1
# anb=np.logical_xor(img,c)*3
# bna=np.logical_xor(testmask,c)*2
# f=c+anb+bna
# dice = np.sum(img[testmask==1])*2.0 / (np.sum(img) + np.sum(testmask))
# plt.imshow(f,cmap = newcmp)
# print(dice)